## translation of search_u_avg.m

2/8/2021


In [1]:
import numpy as numpy 
#can't use np since that is used as a variable for length within the function

from mpmath import *
mp.dps = 15; mp.pretty = True
# allows us to use the exponential integral function ei()

# We need something to compute the lambert function in python
from scipy.special import lambertw # imports the lambert function and returns complex result

import scipy.special as sc
#sc.expi()

In [2]:
# function settl_vel = settling_velocity(rhos,rhog,diam,grav,kin_visc)

def settling_velocity(rhos,rhog,diam,grav,kin_visc):
    inv_sqrt_C_D = 1.0
    for i in range(0,len(xs)):
        
        if rhos[i] <= rhog: 
           settl_vel = 0.0
           return

        const_part =  numpy.sqrt( 0.75 * ( rhos / rhog - 1.0 ) * diam * grav )

        settl_vel = const_part * inv_sqrt_C_D

        Rey = diam * settl_vel / kin_visc

        for i in range(0):

            set_vel_old = settl_vel
            inv_sqrt_C_D_old = inv_sqrt_C_D

            inv_sqrt_C_D = numpy.sqrt( Rey / ( 24.0 * ( 1.0 + 0.15*Rey**0.687 ) ) )
            settl_vel = const_part * inv_sqrt_C_D;
            Rey = diam * settl_vel / kin_visc;
        return(settl_vel)
    

In [3]:
# THINGS TO DO: 
# 1) compute also int(u*u*alpha) (IF NOT POSSIBLE ANALYTICALLY, WITH
#    QUADRATURE): DONE WITH GAUSSIAN QUADRATURE
# 2) correct factor 0.6 by computing the correct value with non-singular
#    velocity profile (SEE MIEDEMA 5.3.44)
# 3) compute ratio of concentration at z=0 and average concentration (DONE)

rel_tol = 1.e-10 
abs_tol = 1.e-8 

# this (when multiplied by h) is one of the conservative variable of the
# system of PDEs.
uRho_avg = 10.0     # depth-averaged u(z)*rhom(z)  [kg/(m2 s)]

# computed by dividing the first conservative variable by mixture density
h = 10.00            # flow thickness [m]

# solid mass fractions (computed from ratio of conservative variables)
xs = numpy.zeros(2)
xs[0] = 0.65 
xs[1] = 0.25 

np = len(xs) 

# gas density can be computed from mixture temperature, which is obtained
# from the conservative variables (T = (h*rhom*Cv*T)/(Cv*h*rhom) )
rhog = 1.1           # [kg/m3]
kin_visc = 1.48e-5   # [m2/s]
grav = 9.81          # [m/s2]

# particles densities [kg/m3]
rhos = numpy.zeros(2)
rhos[0] = 1000 
rhos[1] = 2000 

# uRho_avg_new = 0

# particles diameters [m]
diam = numpy.zeros(2)
diam[0] = 1.e-3 
diam[1] = 1.e-4 

# particles settling velocities [m/s]
settl_vel = settling_velocity(rhos,rhog,diam,grav,kin_visc) 

# fixed parameters
k = 0.4                 # Von Karman constant
friction_coeff = 0.01   # friction coefficient
k_s = 1.5               # substrate roughness [m] 
beta = 1.0              # ratio of momentum and mass diffusivity (Schmidt number)


# gas mass fraction
xg = 1.0 - sum(xs) 

# mixture density
rhom = 1.0/ ( xg/rhog + sum(xs/rhos) ) 

print('Mixture density computed from mass fractions = ',rhom,'kg/m3')

# depth-averaged volumetric fractions
alpha_avg = xs/rhos*rhom     

for i in range(0,np):
    print('Depth-averaged alpha(1) computed from mass fractions = ','[',i+1,']',alpha_avg[i])
    
    
print('Total solid depth-averaged volume fraction computed from mass fractions = ', sum(alpha_avg))

# this is the depth-averaged value of rhos*alpha(z), which is equal to the
# depth-averaged value of xs*rhom(z). So it is the conservative variable
# used for the solid mass divided by h.
rhoAlpha_avg = rhos*alpha_avg 

Mixture density computed from mass fractions =  10.90701767432637 kg/m3
Depth-averaged alpha(1) computed from mass fractions =  [ 1 ] 0.00708956148831214
Depth-averaged alpha(1) computed from mass fractions =  [ 2 ] 0.0013633772092907963
Total solid depth-averaged volume fraction computed from mass fractions =  0.008452938697602937


In [4]:
# H_crit is a critical thickness above which there is a constant free
# stream velocity. Below H_crit we only have the log profile.
# H_crit is a function of k, k_s and friction_coeff only.
# H_crit_rel is the non-dimensional crtitical thickness, which depends on k
# and the friction coefficient only

a = k/numpy.sqrt(friction_coeff)+1.0 
H_crit_rel = numpy.real(1.0/30.0 * ( -a / lambertw(-a*numpy.exp(-a)) -1.0 ))
print('Log region thickness = ',H_crit_rel*k_s,'meters') 

Log region thickness =  7.11624607970279 meters


## Everything above looks good, and outputs as expected

In [5]:
def avg_profiles_mix(h,settl_vel,k,friction_coeff,rhoalpha_avg,u_guess,beta,h0,b,u_coeff,u_rel0,rhos,rhog):

    u_star        = u_guess * numpy.sqrt(friction_coeff)
    Pn            = settl_vel / ( k * u_star )
    np            = len(settl_vel)
    rho_u_alphas  = numpy.zeros(np)
    rhoalphas_int = numpy.zeros(np)

    for i in [0,1]:

        a              = -(6.0*Pn[i]*beta)/h
        int_           = ( ( numpy.exp(a*h0) -1.0 )/a + numpy.exp(a*h0)*(h-h0) )/h
        alphas_rel_max = 1.0/int_
        alphas_rel0 = alphas_rel_max * numpy.exp(a*h0)

    
        rhoalphas_int[i] = rhoalpha_avg[i] * ( alphas_rel_max * ( numpy.exp(a*h0) - 1.0 ) / a + (h-h0)*alphas_rel0 ) / h  

    
        y = h0
        int_h0 = ( numpy.exp(a*y)*numpy.log(b*y+1.0) + numpy.exp(-a/b) * ei( -a*(y+1.0/b) ) ) / a
    
        y = 0
        int_0 = ( numpy.exp(a*y)*numpy.log(b*y+1.0) + numpy.exp(-a/b) * ei( -a*(y+1.0/b) ) ) / a

        int_def = u_coeff * numpy.sqrt(friction_coeff) / k * alphas_rel_max * (int_h0-int_0)

        rho_u_alphas[i] = rhoalpha_avg[i] * u_guess * ( int_def + ( h - h0 ) * alphas_rel0 * u_rel0 ) / h
    
    rhom_avg = rhog + sum((rhos-rhog)/rhos*rhoalphas_int)
    uRho_avg_new = ( u_guess*rhog + sum((rhos-rhog)/rhos*rho_u_alphas) )
    return([rhom_avg,uRho_avg_new])

In [6]:
# The profile parameters depend on h/k_s, not on the absolute value of h.
h_rel = h/k_s 

if (h_rel > H_crit_rel):

    # we search for h0_rel such that the average integral between 0 and
    # h_rel is equal to 1
    # For h_rel > H_crit_rel this integral is the sum of two pieces:
    # integral between 0 and h0_rel of the log profile
    # integral between h0_rel and h_rel of the costant profile

    a = h_rel * k/numpy.sqrt(friction_coeff) 
    b = 1.0/30.0 + h_rel 
    c = 30.0 

    # solve b*log(c*z+1)-z=a for z
    d = a/b-1.0/(b*c) 

    h0_rel = numpy.real(-b*lambertw( -numpy.exp(d)/(b*c) ) - 1.0 / c )
    u_coeff = 1.0 

else:

    # when h_rel <= H_crit_rel we have only the log profile and we have to
    # rescale it in order to have the integral between o and h_rel equal to 1
    # The factor used to scale the velocity is u_coeff

    h0_rel = h_rel 
    u_coeff = k/numpy.sqrt(friction_coeff)/( ( 1 + 1/(30*h_rel) )*numpy.log( 30.0*h_rel + 1.0 )-1.0) 

In [7]:
h0 = h0_rel*k_s 

b = 30.0/k_s 

u_rel0 = u_coeff * numpy.sqrt(friction_coeff) / k * numpy.log( b*h0 + 1.0 ) 

# However Something odd happening in the following cell: 

In [22]:
# search convergence of u_avg
n = 200 

u_avg_guess = uRho_avg / rhom 
x0 = u_avg_guess 
# loop to compute the average velocity from average rho*alpha and average 
# uRho ( = 1/h*int( u*rhog*alphag + sum[u*rhos(i)*alphas(i)] ) ) 

#     avg_profiles_mix function returns ([rhom_avg,uRho_avg_new])

for i in range(0,n):
    x0 = u_avg_guess 
    
    uRho_avg_new = avg_profiles_mix(h,settl_vel,k,friction_coeff,rhoAlpha_avg,u_avg_guess,beta,h0,b,u_coeff,u_rel0,rhos,rhog)[1] 

    u_avg_new = u_avg_guess * uRho_avg / ( uRho_avg_new) 

    x1 = u_avg_new 

    rhom_avg = avg_profiles_mix(h,settl_vel,k,friction_coeff,rhoAlpha_avg,u_avg_new,beta,h0,b,u_coeff,u_rel0,rhos,rhog)[0]
    uRho_avg_new = avg_profiles_mix(h,settl_vel,k,friction_coeff,rhoAlpha_avg,u_avg_new,beta,h0,b,u_coeff,u_rel0,rhos,rhog)[1] 

    u_avg_new = u_avg_new * uRho_avg / ( uRho_avg_new) 
     
    x2 = u_avg_new 

    if (x1 != x0):
        lambda_ = abs((x2 - x1)/(x1 - x0))   #OPTIONAL: Computes an approximation of |f'(fixedPoint)|, which is denoted by lambda

    denominator = (x2 - x1) - (x1 - x0) 
    
    if (abs(denominator) < 0.1*abs_tol):        #To avoid greatly increasing error, do not divide by too small of a number
        print('WARNING: denominator is too small')
        break                                        #Leave the loop

    aitkenX = x2 - ( (x2 - x1)**2 )/denominator 

    u_avg_new = aitkenX    

    if ( abs(u_avg_guess-u_avg_new)/u_avg_guess < rel_tol ) or ( abs(u_avg_guess-u_avg_new) < abs_tol ):
        u_avg_guess = u_avg_new 
        
    u_avg_guess = u_avg_new 

    print('Iteration = ',i, '   u_guess = ',(u_avg_new), ' m/s')

Iteration =  0    u_guess =  nan  m/s
Iteration =  1    u_guess =  nan  m/s
Iteration =  2    u_guess =  nan  m/s
Iteration =  3    u_guess =  nan  m/s
Iteration =  4    u_guess =  nan  m/s
Iteration =  5    u_guess =  nan  m/s
Iteration =  6    u_guess =  nan  m/s
Iteration =  7    u_guess =  nan  m/s
Iteration =  8    u_guess =  nan  m/s
Iteration =  9    u_guess =  nan  m/s
Iteration =  10    u_guess =  nan  m/s
Iteration =  11    u_guess =  nan  m/s
Iteration =  12    u_guess =  nan  m/s
Iteration =  13    u_guess =  nan  m/s
Iteration =  14    u_guess =  nan  m/s
Iteration =  15    u_guess =  nan  m/s
Iteration =  16    u_guess =  nan  m/s
Iteration =  17    u_guess =  nan  m/s
Iteration =  18    u_guess =  nan  m/s
Iteration =  19    u_guess =  nan  m/s
Iteration =  20    u_guess =  nan  m/s
Iteration =  21    u_guess =  nan  m/s
Iteration =  22    u_guess =  nan  m/s
Iteration =  23    u_guess =  nan  m/s
Iteration =  24    u_guess =  nan  m/s
Iteration =  25    u_guess =  nan  

/Users/brandonkeim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':
/Users/brandonkeim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  
/Users/brandonkeim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
/Users/brandonkeim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp
/Users/brandonkeim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/Users/brandonkeim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in exp


In [15]:
print('Total Iterations   = ', i)
print('u_final            = ',u_avg_new,'m/s') 
print('Depth-averaged rho = ' ,rhom_avg ,'kg/m3')

# average velocity
u_avg = u_avg_new 

# shear velocty
u_star = u_avg * numpy.sqrt(friction_coeff) 

# Rouse numbers for different particles
Pn = settl_vel / ( k * u_star ) 

Total Iterations   =  199
u_final            =  4.7843309946899995 m/s
Depth-averaged rho =  10.907017674326367 kg/m3


In [19]:
## Check for average values, computed from u_avg and alpha_avg(i) 

# average velocity computed as integral of the profile
U_int = ( u_avg * ( u_coeff * numpy.sqrt(friction_coeff) / k * ( (h0 + 1.0/b)*numpy.log(b*h0+1)-h0 ) + (h-h0)*u_rel0 ) ) / h 

print('Depth-averaged u              = ',U_int,'m/s') 

print('(rho*u)_avg / (rho_avg*u_avg) = ', uRho_avg/(rhom_avg*U_int)) 

uAlpha_int    = numpy.zeros(np) 
uRhoAlpha_int = numpy.zeros(np) 

for i in range(0,np):

    a = -(6.0*Pn[i]*beta)/h 

    int = ( ( numpy.exp(a*h0) -1.0 )/a + numpy.exp(a*h0)*(h-h0) )/h 

    alpha_rel_max = 1.0/int 

    alpha_rel0 = alpha_rel_max * numpy.exp(a*h0) 

    # average concentration of particle class i
    alpha_int = alpha_avg[i] * ( alpha_rel_max * ( numpy.exp(a*h0) - 1.0 ) / a + (h-h0)*alpha_rel0 ) / h   

    print('Depth-averaged alphas',i,'      = ' , alpha_int)
    
    y = h0 
    int_h0 = ( numpy.exp(a*y)*numpy.log(b*y+1.0) + numpy.exp(-a/b) * ei( -a*(y+1.0/b) ) ) / a 
    y = 0 
    int_0 =  ( numpy.exp(a*y)*numpy.log(b*y+1.0) + numpy.exp(-a/b) * ei( -a*(y+1.0/b) ) ) / a 

    int_def = u_coeff * numpy.sqrt(friction_coeff) / k * alpha_rel_max * (int_h0-int_0) 

    # average of u*alphas(i)
    uAlpha_int[i] = u_avg * alpha_avg[i] * ( int_def + ( h - h0 ) * alpha_rel0 * u_rel0 ) / h 
    uRhoAlpha_int[i] = rhos[i] * alpha_avg[i] * u_avg * ( int_def + ( h - h0 ) * alpha_rel0 * u_rel0 ) / h 
    

uAlpha_int_tot = sum(uAlpha_int) 

rho_u_int_tot = rhog*U_int + sum((rhos-rhog)/rhos*uRhoAlpha_int) 


print('Depth-averaged u*alpha        = ',uAlpha_int_tot)

uRhoAlpha_int_tot = sum(uRhoAlpha_int) 
print('Depth-averaged u*rho          = ', rho_u_int_tot, 'kg m-2 s-1')

Depth-averaged u              =  4.7843309946899995 m/s
(rho*u)_avg / (rho_avg*u_avg) =  0.19163408846680682
Depth-averaged alphas 0       =  0.007089561488312139
Depth-averaged alphas 1       =  0.0013633772092907963
Depth-averaged u*alpha        =  -30686345130.859653
Depth-averaged u*rho          =  -30652590389105.816 kg m-2 s-1


1.89511781635594
